In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Mobile-Price-Prediction-cleaned_data.csv')

In [ ]:
df

,Ratings,RAM,ROM,Mobile_Size,Primary_Cam,Selfi_Cam,Battery_Power,Price
0,4.3,4.0,128.0,6.00,48,13.0,4000,24999
1,3.4,6.0,64.0,4.50,48,12.0,4000,15999
2,4.3,4.0,4.0,4.50,64,16.0,4000,15000
3,4.4,6.0,64.0,6.40,48,15.0,3800,18999
4,4.5,6.0,128.0,6.18,35,15.0,3800,18999
...,...,...,...,...,...,...,...,...
802,3.8,6.0,32.0,4.54,48,12.0,2800,1299
803,4.1,8.0,64.0,4.54,64,8.0,2500,1390
804,4.4,3.0,32.0,6.20,48,1.0,3800,9790
805,3.7,10.0,32.0,4.50,64,8.0,3500,799


In [ ]:
df.drop(columns="Ratings",inplace = True)

In [ ]:
df

,RAM,ROM,Mobile_Size,Primary_Cam,Selfi_Cam,Battery_Power,Price
0,4.0,128.0,6.00,48,13.0,4000,24999
1,6.0,64.0,4.50,48,12.0,4000,15999
2,4.0,4.0,4.50,64,16.0,4000,15000
3,6.0,64.0,6.40,48,15.0,3800,18999
4,6.0,128.0,6.18,35,15.0,3800,18999
...,...,...,...,...,...,...,...
802,6.0,32.0,4.54,48,12.0,2800,1299
803,8.0,64.0,4.54,64,8.0,2500,1390
804,3.0,32.0,6.20,48,1.0,3800,9790
805,10.0,32.0,4.50,64,8.0,3500,799


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RAM            807 non-null    float64
 1   ROM            807 non-null    float64
 2   Mobile_Size    807 non-null    float64
 3   Primary_Cam    807 non-null    int64  
 4   Selfi_Cam      807 non-null    float64
 5   Battery_Power  807 non-null    int64  
 6   Price          807 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 44.3 KB


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Features and Target
X = df.drop('Price', axis=1)
y = df['Price']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X,y

(      RAM    ROM  Mobile_Size  Primary_Cam  Selfi_Cam  Battery_Power
 0     4.0  128.0         6.00           48       13.0           4000
 1     6.0   64.0         4.50           48       12.0           4000
 2     4.0    4.0         4.50           64       16.0           4000
 3     6.0   64.0         6.40           48       15.0           3800
 4     6.0  128.0         6.18           35       15.0           3800
 ..    ...    ...          ...          ...        ...            ...
 802   6.0   32.0         4.54           48       12.0           2800
 803   8.0   64.0         4.54           64        8.0           2500
 804   3.0   32.0         6.20           48        1.0           3800
 805  10.0   32.0         4.50           64        8.0           3500
 806   6.0   32.0         4.50           64       15.0           1050
 
 [807 rows x 6 columns],
 0      24999
 1      15999
 2      15000
 3      18999
 4      18999
        ...  
 802     1299
 803     1390
 804     9790
 805   

In [ ]:
# Preprocessing
numeric_features = ['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam', 'Battery_Power']
numeric_transformer = ColumnTransformer(
    transformers=[('scaler', StandardScaler(), X.columns)],
    remainder='passthrough'
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])


In [ ]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 ColumnTransformer(remainder='passthrough',
                                                   transformers=[('scaler',
                                                                  StandardScaler(),
                                                                  Index(['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam',
       'Battery_Power'],
      dtype='object'))]),
                                 ['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam',
                                  'Selfi_Cam', 'Battery_Power'])])

In [ ]:
def objective(trial):
    # Define hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        ))
    ])

    # Perform cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    return -scores.mean()


In [ ]:
# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Best hyperparameters
print("Best Hyperparameters:", study.best_params)

[I 2025-03-18 16:09:03,770] A new study created in memory with name: no-name-8ac8390f-11ef-4c9b-944d-a8b0044e9cb5
[I 2025-03-18 16:09:05,741] Trial 0 finished with value: 157539481.56287086 and parameters: {'n_estimators': 238, 'max_depth': 29, 'min_samples_split': 10, 'min_samples_leaf': 9}. Best is trial 0 with value: 157539481.56287086.
[I 2025-03-18 16:09:07,910] Trial 1 finished with value: 158800811.12525803 and parameters: {'n_estimators': 269, 'max_depth': 8, 'min_samples_split': 19, 'min_samples_leaf': 9}. Best is trial 0 with value: 157539481.56287086.
[I 2025-03-18 16:09:08,818] Trial 2 finished with value: 133403148.6551387 and parameters: {'n_estimators': 97, 'max_depth': 28, 'min_samples_split': 19, 'min_samples_leaf': 3}. Best is trial 2 with value: 133403148.6551387.
[I 2025-03-18 16:09:10,368] Trial 3 finished with value: 141460043.3801368 and parameters: {'n_estimators': 181, 'max_depth': 14, 'min_samples_split': 11, 'min_samples_leaf': 7}. Best is trial 2 with value:

Best Hyperparameters: {'n_estimators': 177, 'max_depth': 39, 'min_samples_split': 4, 'min_samples_leaf': 1}


In [ ]:
# Train final model with best parameters
best_params = study.best_params
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        **best_params,
        random_state=42
    ))
])

In [ ]:
final_model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('scaler',
                                                                                   StandardScaler(),
                                                                                   Index(['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam',
       'Battery_Power'],
      dtype='object'))]),
                                                  ['RAM', 'ROM', 'Mobile_Size',
                                                   'Primary_Cam', 'Selfi_Cam',
                                                   'Battery_Power'])])),
                ('model',
                 RandomForestRegressor(max_depth=39, min_samples_split=4,
                                       n_estimators=177, random_state=42))])

In [ ]:
# Predictions
y_pred = final_model.predict(X_test)

In [ ]:
y_pred

array([  1429.20341001,    798.36991483,    798.36991483,  18396.9866021 ,
         2220.11957224,  19206.33574791,  13525.15090799,  14402.422424  ,
        20169.15532688,   3007.66545601,  10658.88696529,    871.47531399,
          855.24248628,   1250.18657234,   1186.3775874 ,   1311.40054785,
         1684.66468927,   3550.80036992,    937.52229396,   1486.87769258,
        20707.52615012,   1560.22417348,  29473.25263653,   3384.16548292,
        15652.42834948,  57539.30672137,    798.36991483, 137044.08124832,
        18698.56670702,   3384.16548292,  23933.13544974,  38113.69622455,
        42270.72527576,  17263.69746435,   3081.15304009,   1457.19212402,
          798.36991483,  29473.25263653,  41742.10321047,  33579.32845933,
        10658.88696529,  15023.89844857,   3073.12818622,  16937.88201507,
        40600.07382481,  20416.43736885,  15045.26642454,    974.03398574,
         1965.33700341,   9302.67218635,    803.57849934,   1976.72987176,
        32723.04329208,  

In [ ]:
# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")


Mean Squared Error: 55924366.826044224
R2 Score: 0.8675774246746216


In [ ]:
# Save the model using pickle
import pickle
with open('best_regression_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print("Model saved as best_regression_model.pkl")


Model saved as best_regression_model.pkl


In [ ]:
# Load the model using pickle
with open('best_regression_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# Example Prediction
sample_input = pd.DataFrame({
    'RAM': [8],
    'ROM': [128],
    'Mobile_Size': [6.5],
    'Primary_Cam': [50],
    'Selfi_Cam': [32],
    'Battery_Power': [5000]
})
predicted_price = model.predict(sample_input)
print(f"Predicted Price: {predicted_price[0]}")

Predicted Price: 24153.71818670971


In [ ]:
final_model.named_steps

{'preprocessor': ColumnTransformer(transformers=[('num',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('scaler',
                                                                   StandardScaler(),
                                                                   Index(['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam',
        'Battery_Power'],
       dtype='object'))]),
                                  ['RAM', 'ROM', 'Mobile_Size', 'Primary_Cam',
                                   'Selfi_Cam', 'Battery_Power'])]),
 'model': RandomForestRegressor(max_depth=39, min_samples_split=4, n_estimators=177,
                       random_state=42)}